# The below implementation is taken from [keras for researchers](https://keras.io/getting_started/intro_to_keras_for_researchers/)

In [9]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

## Tensors
TensorFlow is an infrastructure layer for differentiable programming. At its heart, it's a framework for manipulating N-dimensional arrays (tensors), much like NumPy.

However, there are three key differences between NumPy and TensorFlow:

TensorFlow can leverage hardware accelerators such as GPUs and TPUs.
TensorFlow can automatically compute the gradient of arbitrary differentiable tensor expressions.
TensorFlow computation can be distributed to large numbers of devices on a single machine, and large number of machines (potentially with multiple devices each).
Let's take a look at the object that is at the core of TensorFlow: the Tensor.

Here's a constant tensor:



In [10]:
# Here's a constant tensor:
x = tf.constant([[5, 2], [1, 3]])
print(x)

# You can get its value as a NumPy array by calling .numpy():
print(x.numpy())
# Much like a NumPy array, it features the attributes dtype and shape:
print("dtype:", x.dtype)
print("shape:", x.shape)


tf.Tensor(
[[5 2]
 [1 3]], shape=(2, 2), dtype=int32)
[[5 2]
 [1 3]]
dtype: <dtype: 'int32'>
shape: (2, 2)


A common way to create constant tensors is via tf.ones and tf.zeros (just like np.ones and np.zeros):


In [11]:
print(tf.ones(shape=(2, 1)))
print(tf.zeros(shape=(2, 1)))

# You can also create random constant tensors:

x = tf.random.normal(shape=(2, 2), mean=0.0, stddev=1.0)

x = tf.random.uniform(shape=(2, 2), minval=0, maxval=10, dtype="int32")


tf.Tensor(
[[1.]
 [1.]], shape=(2, 1), dtype=float32)
tf.Tensor(
[[0.]
 [0.]], shape=(2, 1), dtype=float32)


## Variables
Variables are special tensors used to store mutable state (such as the weights of a neural network). You create a Variable using some initial value:

In [12]:
initial_value = tf.random.normal(shape=(2, 2))
a = tf.Variable(initial_value)
print(a)

<tf.Variable 'Variable:0' shape=(2, 2) dtype=float32, numpy=
array([[ 0.5958199 ,  0.37926623],
       [-1.0933207 , -0.73762554]], dtype=float32)>


You update the value of a Variable by using the methods .assign(value), .assign_add(increment), or .assign_sub(decrement):



In [13]:
new_value = tf.random.normal(shape=(2, 2))
a.assign(new_value)
for i in range(2):
    for j in range(2):
        assert a[i, j] == new_value[i, j]

added_value = tf.random.normal(shape=(2, 2))
a.assign_add(added_value)
for i in range(2):
    for j in range(2):
        assert a[i, j] == new_value[i, j] + added_value[i, j]


# Gradients
Here's another big difference with NumPy: you can automatically retrieve the gradient of any differentiable expression.

Just open a GradientTape, start "watching" a tensor via tape.watch(), and compose a differentiable expression using this tensor as input:



In [14]:
a = tf.random.normal(shape=(2, 2))
b = tf.random.normal(shape=(2, 2))

with tf.GradientTape() as tape:
    tape.watch(a)  # Start recording the history of operations applied to `a`
    c = tf.sqrt(tf.square(a) + tf.square(b))  # Do some math using `a`
    # What's the gradient of `c` with respect to `a`?
    dc_da = tape.gradient(c, a)
    print(dc_da)


tf.Tensor(
[[ 0.70634687 -0.87136996]
 [ 0.48079547 -0.63041496]], shape=(2, 2), dtype=float32)


Note that you can compute higher-order derivatives by nesting tapes:



In [19]:
a = tf.random.normal(shape=(2, 2),dtype=tf.float32)
b = tf.random.normal(shape=(2, 2),dtype=tf.float32)

with tf.GradientTape() as outer_tape:
    with tf.GradientTape() as tape:
        c = tf.sqrt(tf.square(a) + tf.square(b))
        dc_da = tape.gradient(c, a)
    
    d2c_da2 = outer_tape.gradient(dc_da, a)
    print(d2c_da2)


TypeError: Cannot convert value None to a TensorFlow DType.

# Keras layers
While TensorFlow is an infrastructure layer for differentiable programming, dealing with tensors, variables, and gradients, Keras is a user interface for deep learning, dealing with layers, models, optimizers, loss functions, metrics, and more.

Keras serves as the high-level API for TensorFlow: Keras is what makes TensorFlow simple and productive.

The Layer class is the fundamental abstraction in Keras. A Layer encapsulates a state (weights) and some computation (defined in the call method).

A simple layer looks like this:

